In [1]:
#import the libraries
import pandas as pd
import numpy as np
import squarify as sq
from datetime import datetime
from fim import arules
import warnings
from operator import attrgetter

In [2]:
# Upload the csv files 
fact = pd.read_csv("fact_table.csv",encoding='latin1') # encoding='latin1' to avoid warring
item = pd.read_csv("item_dim.csv",encoding='latin1')
store = pd.read_csv("store_dim.csv",encoding='latin1')
time = pd.read_csv('time_dim.csv',encoding='latin1')
transactions = pd.read_csv('Trans_dim.csv',encoding='latin1')
customers = pd.read_csv("customer_dim.csv", encoding='latin1')

In [3]:
fact.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0


In [4]:
item.head()

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans


In [5]:
store.head()

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR


In [6]:
time.head()

,time_key,date,hour,day,week,month,quarter,year
0,T00001,20-05-2017 14:56,14,20,3rd Week,5,Q2,2017
1,T00002,30-01-2015 22:14,22,30,4th Week,1,Q1,2015
2,T00003,14-03-2020 02:34,2,14,2nd Week,3,Q1,2020
3,T00004,27-04-2018 12:19,12,27,4th Week,4,Q2,2018
4,T00005,14-04-2018 10:43,10,14,2nd Week,4,Q2,2018


In [7]:
transactions.head()

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited


In [8]:
customers.head()

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842
2,C000003,kailash kumar,8801663795774,3769494056318
3,C000004,bhagwati prasad,8801533627961,9378834712725
4,C000005,ajay,8801943715786,3540815556323


### EDA

In [9]:
# check the types to see for changes and to merge the dataframes

In [10]:
fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   payment_key    1000000 non-null  object 
 1   coustomer_key  1000000 non-null  object 
 2   time_key       1000000 non-null  object 
 3   item_key       1000000 non-null  object 
 4   store_key      1000000 non-null  object 
 5   quantity       1000000 non-null  int64  
 6   unit           996277 non-null   object 
 7   unit_price     1000000 non-null  float64
 8   total_price    1000000 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 68.7+ MB


In [11]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_key     264 non-null    object 
 1   item_name    264 non-null    object 
 2   desc         264 non-null    object 
 3   unit_price   264 non-null    float64
 4   man_country  264 non-null    object 
 5   supplier     264 non-null    object 
 6   unit         263 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.6+ KB


In [12]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_key  726 non-null    object
 1   division   726 non-null    object
 2   district   726 non-null    object
 3   upazila    726 non-null    object
dtypes: object(4)
memory usage: 22.8+ KB


In [13]:
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   time_key  99999 non-null  object
 1   date      99999 non-null  object
 2   hour      99999 non-null  int64 
 3   day       99999 non-null  int64 
 4   week      99999 non-null  object
 5   month     99999 non-null  int64 
 6   quarter   99999 non-null  object
 7   year      99999 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 6.1+ MB


In [14]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   payment_key  39 non-null     object
 1   trans_type   39 non-null     object
 2   bank_name    39 non-null     object
dtypes: object(3)
memory usage: 1.0+ KB


In [15]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9191 entries, 0 to 9190
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   coustomer_key  9191 non-null   object
 1   name           9164 non-null   object
 2   contact_no     9191 non-null   int64 
 3   nid            9191 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 287.3+ KB


In [16]:
#change the type of date from object to datetime
time['date'] = pd.to_datetime(time['date'], format='%d-%m-%Y %H:%M')
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   time_key  99999 non-null  object        
 1   date      99999 non-null  datetime64[ns]
 2   hour      99999 non-null  int64         
 3   day       99999 non-null  int64         
 4   week      99999 non-null  object        
 5   month     99999 non-null  int64         
 6   quarter   99999 non-null  object        
 7   year      99999 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 6.1+ MB


In [17]:
#In the fact table we have Nulls 
fact.isnull().sum()

payment_key         0
coustomer_key       0
time_key            0
item_key            0
store_key           0
quantity            0
unit             3723
unit_price          0
total_price         0
dtype: int64

In [18]:
# merge all the tables
merged_fact = pd.merge(fact,item[["item_key","item_name","man_country","supplier","desc"]], on='item_key')
merged_fact = pd.merge(merged_fact,customers, on = "coustomer_key")
merged_fact = pd.merge(merged_fact,store, on = "store_key")
merged_fact = pd.merge(merged_fact,transactions,on = "payment_key")
merged_fact =  pd.merge(merged_fact,time , on = "time_key")
merged_fact.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,item_name,...,upazila,trans_type,bank_name,date,hour,day,week,month,quarter,year
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,...,ISLAMPUR,card,Premier Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016
1,P002,C002344,T049189,I00058,S00677,1,bottles,22.0,22.0,Premier Protein Shake Choc. 11oz,...,SUTRAPUR,card,AB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016
2,P004,C002005,T049189,I00101,S00408,2,tubs,15.0,30.0,French Vanilla International Delight,...,ROWANGCHHARI,card,Bank Asia Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016
3,P035,C007222,T049189,I00042,S00616,3,bottles,28.0,84.0,Topo Chico 12 oz glass,...,GULSHAN,card,Uttara Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016
4,P021,C001803,T049189,I00036,S00119,8,cans,16.0,128.0,La Croix Sparkling Orange 12 oz,...,NAOGAON SADAR,card,NRB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016


In [19]:
#Categorizing the time of the order to:7-13=morning ,13-19=noon,19-22=afternoon,22-7=night
def categorize_time(time):
    if time>=0 and time<=6:
        return 'night'
    elif time>=7 and time<=12:
        return 'morning'
    elif time>=13 and time<=18:
        return 'noon'
    elif time>=19 and time<=21:
        return 'afternoon'
    else:
        return "night"
merged_fact['time_category']=merged_fact['hour'].map(categorize_time)
merged_fact.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,item_name,...,trans_type,bank_name,date,hour,day,week,month,quarter,year,time_category
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,...,card,Premier Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon
1,P002,C002344,T049189,I00058,S00677,1,bottles,22.0,22.0,Premier Protein Shake Choc. 11oz,...,card,AB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon
2,P004,C002005,T049189,I00101,S00408,2,tubs,15.0,30.0,French Vanilla International Delight,...,card,Bank Asia Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon
3,P035,C007222,T049189,I00042,S00616,3,bottles,28.0,84.0,Topo Chico 12 oz glass,...,card,Uttara Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon
4,P021,C001803,T049189,I00036,S00119,8,cans,16.0,128.0,La Croix Sparkling Orange 12 oz,...,card,NRB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon


In [20]:
#Categorizing the country where the item is manufactured in continents
def categorize_continent(man_country):
    if man_country in ["Netherlands", "Germany", "Finland", "Lithuania", "Poland"]:
        return 'Europe'
    elif man_country == "United States":
        return "North America"
    else:
        return "Asia"

merged_fact['continent'] = merged_fact['man_country'].map(categorize_continent)

merged_fact.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,item_name,...,bank_name,date,hour,day,week,month,quarter,year,time_category,continent
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,...,Premier Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe
1,P002,C002344,T049189,I00058,S00677,1,bottles,22.0,22.0,Premier Protein Shake Choc. 11oz,...,AB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America
2,P004,C002005,T049189,I00101,S00408,2,tubs,15.0,30.0,French Vanilla International Delight,...,Bank Asia Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe
3,P035,C007222,T049189,I00042,S00616,3,bottles,28.0,84.0,Topo Chico 12 oz glass,...,Uttara Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America
4,P021,C001803,T049189,I00036,S00119,8,cans,16.0,128.0,La Croix Sparkling Orange 12 oz,...,NRB Bank Limited,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Asia


In [21]:
merged_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 30 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   payment_key    1000000 non-null  object        
 1   coustomer_key  1000000 non-null  object        
 2   time_key       1000000 non-null  object        
 3   item_key       1000000 non-null  object        
 4   store_key      1000000 non-null  object        
 5   quantity       1000000 non-null  int64         
 6   unit           996277 non-null   object        
 7   unit_price     1000000 non-null  float64       
 8   total_price    1000000 non-null  float64       
 9   item_name      1000000 non-null  object        
 10  man_country    1000000 non-null  object        
 11  supplier       1000000 non-null  object        
 12  desc           1000000 non-null  object        
 13  name           997010 non-null   object        
 14  contact_no     1000000 non-null  in

In [22]:
#remοve nulls from the merged_fact table
merged_fact= merged_fact.dropna()
merged_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 993306 entries, 0 to 999999
Data columns (total 30 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   payment_key    993306 non-null  object        
 1   coustomer_key  993306 non-null  object        
 2   time_key       993306 non-null  object        
 3   item_key       993306 non-null  object        
 4   store_key      993306 non-null  object        
 5   quantity       993306 non-null  int64         
 6   unit           993306 non-null  object        
 7   unit_price     993306 non-null  float64       
 8   total_price    993306 non-null  float64       
 9   item_name      993306 non-null  object        
 10  man_country    993306 non-null  object        
 11  supplier       993306 non-null  object        
 12  desc           993306 non-null  object        
 13  name           993306 non-null  object        
 14  contact_no     993306 non-null  int64         
 15  

In [23]:
#drop the columns that we dont need
merged_fact = merged_fact.drop(columns=["payment_key","time_key","store_key","item_key","nid"]) 

### RFM

In [24]:
rfm_df = merged_fact

Create RFM Table
For each customer find:

Recency : minimum Recency

Frequency: count unique number of days of purchases

Monetary: summary of Total Sales

In [25]:
rfm_df["date"].max() # see the last date we have in this dataframe

Timestamp('2021-01-23 04:38:00')

In [26]:
# create the recency
rfm_df['Recency'] = (rfm_df['date'].max() - rfm_df['date']).dt.days 

rfm_df.head()

,coustomer_key,quantity,unit,unit_price,total_price,item_name,man_country,supplier,desc,name,...,date,hour,day,week,month,quarter,year,time_category,continent,Recency
0,C004510,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,Netherlands,Bolsius Boxmeer,Food - Chocolate,priya,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe,1732
1,C002344,1,bottles,22.0,22.0,Premier Protein Shake Choc. 11oz,United States,MAESA SAS,Beverage - Energy/Protein,priti,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America,1732
2,C002005,2,tubs,15.0,30.0,French Vanilla International Delight,Germany,Friedola 1888 GmbH,Coffee Cream,payal,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe,1732
3,C007222,3,bottles,28.0,84.0,Topo Chico 12 oz glass,United States,MAESA SAS,a. Beverage Sparkling Water,parbha devi,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America,1732
4,C001803,8,cans,16.0,128.0,La Croix Sparkling Orange 12 oz,China,"CHERRY GROUP CO.,LTD",a. Beverage Sparkling Water,sita devi,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Asia,1732


In [27]:
# create data_processed 
data_processed = rfm_df.groupby('coustomer_key').agg({'Recency' : 'min',
                          'date' : pd.Series.nunique, # Count the number of unique values 
                          'total_price' : 'sum'})

In [28]:
data_processed

,Recency,date,total_price
coustomer_key,,,
C000001,10,82,8344.75
C000002,61,103,11025.50
C000003,81,108,12120.25
C000004,32,116,13218.00
C000005,60,107,10706.25
...,...,...,...
C009187,27,109,11800.75
C009188,25,118,13868.75
C009189,0,114,11514.50


In [29]:
#rename the columns 
data_processed.rename(columns={'Recency':'Recency', 'date':'Frequency', 'total_price':'Monetary'}, inplace=True) 
data_processed.head()

,Recency,Frequency,Monetary
coustomer_key,,,
C000001,10,82,8344.75
C000002,61,103,11025.50
C000003,81,108,12120.25
C000004,32,116,13218.00
C000005,60,107,10706.25


In [30]:
# Create Scores RFM scores based on quantiles of distribution

#Date from customer's last purchase.The nearest date gets 4 and the furthest date gets 1.
data_processed['recency_score'] = pd.qcut(data_processed['Recency'], 4, labels=[4,3,2,1])

# Total number of days with purchase. The lowest frequency gets 1 and the highest gets 4
data_processed['frequency_score'] = pd.qcut(data_processed['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])

# Total spend of each customer. The lowest amount gets 1 and the highest gets 4
data_processed['monetary_score'] = pd.qcut(data_processed['Monetary'], 4, labels=[1,2,3,4])

data_processed.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
coustomer_key,,,,,,
C000001,10,82,8344.75,3,1,1
C000002,61,103,11025.50,1,2,2
C000003,81,108,12120.25,1,2,3
C000004,32,116,13218.00,2,4,4
C000005,60,107,10706.25,1,2,2


In [31]:
#create the rfm_segment
data_processed['RFM_Segment'] = data_processed['recency_score'].astype(str)+data_processed['frequency_score'].astype(str)+data_processed['monetary_score'].astype(str)
data_processed

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment
coustomer_key,,,,,,,
C000001,10,82,8344.75,3,1,1,311
C000002,61,103,11025.50,1,2,2,122
C000003,81,108,12120.25,1,2,3,123
C000004,32,116,13218.00,2,4,4,244
C000005,60,107,10706.25,1,2,2,122
...,...,...,...,...,...,...,...
C009187,27,109,11800.75,2,3,3,233
C009188,25,118,13868.75,2,4,4,244
C009189,0,114,11514.50,4,3,3,433


In [32]:
# sum the rows (recency_score,frequency_score,monetary_score) to get Rfm_score
data_processed['RFM_Score'] = data_processed[['recency_score', 'frequency_score', 'monetary_score' ]].sum(axis=1)
data_processed

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score
coustomer_key,,,,,,,,
C000001,10,82,8344.75,3,1,1,311,5
C000002,61,103,11025.50,1,2,2,122,5
C000003,81,108,12120.25,1,2,3,123,6
C000004,32,116,13218.00,2,4,4,244,10
C000005,60,107,10706.25,1,2,2,122,5
...,...,...,...,...,...,...,...,...
C009187,27,109,11800.75,2,3,3,233,8
C009188,25,118,13868.75,2,4,4,244,10
C009189,0,114,11514.50,4,3,3,433,10


In [33]:
# Use regex and dictionaries values to categorize numbers from customers
segt_map = {
    r'[3-4][3-4]4': 'VIP',
    r'[2-3-4][1-2-3-4]4': 'Top Recent',
    r'1[1-2-3-4]4': 'Top at Risk ',

    
    
    r'[3-4][3-4]3': 'High Promising',
    r'[2-3-4][1-2]3': 'High New',
    r'2[3-4]3': 'High Loyal',

    
    
    r'[3-4][3-4]2': 'Medium Potential',
    r'[2-3-4][1-2]2': 'Medium New',
    r'2[3-4]2': 'Medium Loyal',

    
    
    r'4[1-2-3-4]1': 'Low New',
    r'[2-3][1-2-3-4]1': 'Low Loyal',
    
    r'1[1-2-3-4][1-2-3]': 'Need Activation'
}
data_processed['Segment_labels'] = data_processed['RFM_Segment']
data_processed['Segment_labels'] = data_processed['Segment_labels'].replace(segt_map, regex=True)
data_processed.reset_index(inplace=True)
data_processed.head()

,coustomer_key,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score,Segment_labels
0,C000001,10,82,8344.75,3,1,1,311,5,Low Loyal
1,C000002,61,103,11025.50,1,2,2,122,5,Need Activation
2,C000003,81,108,12120.25,1,2,3,123,6,Need Activation
3,C000004,32,116,13218.00,2,4,4,244,10,Top Recent
4,C000005,60,107,10706.25,1,2,2,122,5,Need Activation


## Calculate Customers per Segments and all Stats per Segment

In [34]:
#create segments_grouped
segements_grouped  = data_processed.groupby(["Segment_labels"]).agg({'Monetary': np.sum,
                                   "coustomer_key": pd.Series.nunique,# Count the number of unique values 
                                    "Recency": np.mean,                     
                                     "Frequency": np.mean,
                                                        }).reset_index()

In [35]:
#Create the persentages and rename the columns
segements_grouped["Monetary%"] = segements_grouped["Monetary"] / segements_grouped["Monetary"].sum()*100
segements_grouped["Count%"] = segements_grouped["coustomer_key"] / segements_grouped["coustomer_key"].sum()*100
segements_grouped.rename(columns={'Monetary':'TotalMonetary','CustomerID':'TotalCustomers',
                                 'Recency':'MeanRecency','Frequency':'MeanFrequency',
                                 "Count%":"Customers%"},inplace= True)
segements_grouped = segements_grouped.sort_values(by=['Monetary%'], ascending=False)
segements_grouped

,Segment_labels,TotalMonetary,coustomer_key,MeanRecency,MeanFrequency,Monetary%,Customers%
8,Need Activation,18752702.00,1741,56.679495,104.118897,17.909947,18.998254
11,VIP,14625333.00,1103,7.248413,120.134180,13.968064,12.036229
6,Medium New,12194619.25,1119,12.716711,102.126899,11.646588,12.210825
3,Low Loyal,10390142.50,1070,18.106542,98.104673,9.923205,11.676124
2,High Promising,9335552.25,785,6.964331,114.703185,8.916008,8.566128
9,Top Recent,8804863.25,670,21.044776,116.773134,8.409169,7.311218
1,High New,6972680.50,592,12.391892,103.839527,6.659325,6.460061
10,Top at Risk,6820416.25,518,53.752896,118.544402,6.513904,5.652553
4,Low New,5872882.50,602,3.315615,98.554817,5.608953,6.569184
0,High Loyal,4530224.50,382,23.767016,114.654450,4.326634,4.168485


In [36]:
merged_fact.head()

,coustomer_key,quantity,unit,unit_price,total_price,item_name,man_country,supplier,desc,name,...,date,hour,day,week,month,quarter,year,time_category,continent,Recency
0,C004510,1,ct,35.0,35.0,M&M Peanut Candy 1.7 oz,Netherlands,Bolsius Boxmeer,Food - Chocolate,priya,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe,1732
1,C002344,1,bottles,22.0,22.0,Premier Protein Shake Choc. 11oz,United States,MAESA SAS,Beverage - Energy/Protein,priti,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America,1732
2,C002005,2,tubs,15.0,30.0,French Vanilla International Delight,Germany,Friedola 1888 GmbH,Coffee Cream,payal,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Europe,1732
3,C007222,3,bottles,28.0,84.0,Topo Chico 12 oz glass,United States,MAESA SAS,a. Beverage Sparkling Water,parbha devi,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,North America,1732
4,C001803,8,cans,16.0,128.0,La Croix Sparkling Orange 12 oz,China,"CHERRY GROUP CO.,LTD",a. Beverage Sparkling Water,sita devi,...,2016-04-26 17:13:00,17,26,4th Week,4,Q2,2016,noon,Asia,1732


In [37]:
segements_grouped

,Segment_labels,TotalMonetary,coustomer_key,MeanRecency,MeanFrequency,Monetary%,Customers%
8,Need Activation,18752702.00,1741,56.679495,104.118897,17.909947,18.998254
11,VIP,14625333.00,1103,7.248413,120.134180,13.968064,12.036229
6,Medium New,12194619.25,1119,12.716711,102.126899,11.646588,12.210825
3,Low Loyal,10390142.50,1070,18.106542,98.104673,9.923205,11.676124
2,High Promising,9335552.25,785,6.964331,114.703185,8.916008,8.566128
9,Top Recent,8804863.25,670,21.044776,116.773134,8.409169,7.311218
1,High New,6972680.50,592,12.391892,103.839527,6.659325,6.460061
10,Top at Risk,6820416.25,518,53.752896,118.544402,6.513904,5.652553
4,Low New,5872882.50,602,3.315615,98.554817,5.608953,6.569184
0,High Loyal,4530224.50,382,23.767016,114.654450,4.326634,4.168485


In [38]:
data_processed.head()

,coustomer_key,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score,Segment_labels
0,C000001,10,82,8344.75,3,1,1,311,5,Low Loyal
1,C000002,61,103,11025.50,1,2,2,122,5,Need Activation
2,C000003,81,108,12120.25,1,2,3,123,6,Need Activation
3,C000004,32,116,13218.00,2,4,4,244,10,Top Recent
4,C000005,60,107,10706.25,1,2,2,122,5,Need Activation


### Save as csv

In [39]:
merged_fact.to_csv("merged_fact.csv",index = False)

In [40]:
data_processed.to_csv("data_processed.csv",index = False)

In [41]:
segements_grouped.to_csv("segements_grouped.csv",index = False)